In [ ]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense, Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D
from keras.layers.recurrent import LSTM
from keras.layers.merge import concatenate
import pickle

In [ ]:
infile = open("bytes1.pickle",'rb')
train_bytes = pickle.load(infile)
infile.close()

In [ ]:
infile1 = open("bytes2.pickle",'rb')
test_bytes = pickle.load(infile1)
infile1.close()

In [ ]:
infile2 = open("statistics1.pickle",'rb')
X_train = pickle.load(infile2)
infile2.close()

In [ ]:
infile3 = open("statistics2.pickle",'rb')
X_test = pickle.load(infile3)
infile3.close()

In [ ]:
infile4 = open("timeseries1.pickle",'rb')
new_dict1 = pickle.load(infile4)
infile4.close()

In [ ]:
infile5 = open("timeseries2.pickle",'rb')
new_dict2 = pickle.load(infile5)
infile5.close()

In [ ]:
infile6 = open("labels1.pickle",'rb')
train_labels = pickle.load(infile6)
infile6.close()

In [ ]:
infile7 = open("labels2.pickle",'rb')
test_labels = pickle.load(infile7)
infile7.close()

In [ ]:
visible = a_layer = Input(shape=(76, ) , name="a")
hidden1 = Dense(200, activation="relu")(visible)
output1 = Dense(200, activation="relu")(hidden1)

In [ ]:
visible_l = b_layer = Input(shape=(1024, 3) , name="b")
hidden1_l = Dense(512, activation="relu", name="m1")(visible_l)
hidden2_l = LSTM(256, return_sequences=True)(hidden1_l)
hidden3_l = LSTM(256, return_sequences=True)(hidden2_l)
hidden4_l = LSTM(256)(hidden3_l)

In [ ]:
visible = c_layer = Input(shape=(1800, 1), name="c")
conv1 = Conv1D(256, kernel_size=2, activation='relu')(visible)
conv2 = Conv1D(256, kernel_size=2, activation='relu')(conv1)
pool1 = MaxPooling1D(pool_size=(2,))(conv2)
conv3 = Conv1D(128, kernel_size=2, activation='relu')(pool1)
conv4 = Conv1D(128, kernel_size=2, activation='relu')(conv3)
pool2 = MaxPooling1D(pool_size=(2,))(conv4)
output2 = Flatten()(pool2)

In [ ]:
merge = concatenate([output1, hidden4_l, output2])
hiddeno_1=Dense(128, activation='relu')(merge)
hiddeno_2=Dense(128, activation='relu')(hiddeno_1)
output_final = Dense(8, activation='softmax')(hiddeno_2)
model = Model(inputs=[a_layer, b_layer, c_layer], outputs=output_final)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

In [ ]:
import tensorflow as tf
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
model.fit(x={"a": X_train, "b": new_dict1, "c": train_bytes}, y=train_labels, epochs=40, callbacks=[callback], verbose=1)

In [ ]:
model.evaluate(x={"a": X_test, "b": new_dict2, "c": test_bytes}, y=test_labels)